<a href="https://colab.research.google.com/github/nargesalavi/Quantum-Open-Source-Foundation-Mentorship/blob/master/QNN_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.1.0

In [ ]:
!pip install tensorflow-quantum

In [18]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
qubits = cirq.GridQubit.rect(1, 4)

In [ ]:
def generate_even_block(block_number):
    """ Function for generating the even blocks
        block_number: Block number, it has to be even.
        return: ciq.Circuit """
    params = sympy.symbols(['theta_{}'.format(n) for n in range((block_number-1)*4,block_number*4)])

    # create the parameterized circuit
    circuit = cirq.Circuit(
        cirq.rz(params[0])(qubits[0]),
        cirq.rz(params[1])(qubits[1]),
        cirq.rz(params[2])(qubits[2]),
        cirq.rz(params[3])(qubits[3]),
        cirq.CZ(qubits[0],qubits[1]),
        cirq.CZ(qubits[0],qubits[2]),
        cirq.CZ(qubits[0],qubits[3]),
        cirq.CZ(qubits[1],qubits[2]),
        cirq.CZ(qubits[1],qubits[3]),
        cirq.CZ(qubits[2],qubits[3])
    )
    
    return circuit

In [ ]:
def generate_odd_block(block_number):
    """ Function for generating the odd blocks
        block_number: Block number, it has to be odd.
        return: ciq.Circuit """
    params = sympy.symbols(['theta_{}'.format(n) for n in range((block_number-1)*4,block_number*4)])

    # create the parameterized circuit
    circuit = cirq.Circuit(
        cirq.rx(params[0])(qubits[0]),
        cirq.rx(params[1])(qubits[1]),
        cirq.rx(params[2])(qubits[2]),
        cirq.rx(params[3])(qubits[3])
    )
    
    return circuit

In [ ]:
def generate_qnn(l):
    """ Function for generating qnn, containing l number of layers.
        l: number of layers, each layer contains one odd and one even block.
        return: ciq.Circuit """
    circuit = cirq.Circuit()
    for i in range(1,2*l+1):
        if i % 2 == 1:
            circuit += generate_odd_block(i)
        else:
            circuit += generate_even_block(i)
        
    return circuit

In [ ]:
l=1
qnn = generate_qnn(l)
print(qnn)

                                               ┌──┐
(0, 0): ───Rx(theta_0)───Rz(theta_4)───@───@────@─────────────
                                       │   │    │
(0, 1): ───Rx(theta_1)───Rz(theta_5)───@───┼────┼@────@───────
                                           │    ││    │
(0, 2): ───Rx(theta_2)───Rz(theta_6)───────@────┼@────┼───@───
                                                │     │   │
(0, 3): ───Rx(theta_3)───Rz(theta_7)────────────@─────@───@───
                                               └──┘


In [30]:
params = tf.zeros([1,8*l ], tf.float32)
print(params)
params_names = sympy.symbols(['theta_{}'.format(n) for n in range(8*l)])

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 8), dtype=float32)


In [31]:
state_layer = tfq.layers.State()
state = state_layer(qnn, symbol_names=params_names, symbol_values=params)
print(state)

Instructions for updating:
reduction_indices is deprecated, use axis instead
<tf.RaggedTensor [[(1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j]]>


In [33]:
random_circuit = cirq.testing.random_circuit(qubits = qubits,n_moments = np.random.randint(low=1,high=5),\
                                                                op_density = 0.99999999)
print(random_circuit)
target_state = tfq.layers.State()(random_circuit)
print(target_state)

                   ┌──┐   ┌──────┐
(0, 0): ───iSwap────X───────iSwap────
           │        │       │
(0, 1): ───iSwap────┼──────Y┼────────
                    │       │
(0, 2): ───@────────┼H──────┼────────
           │        │       │
(0, 3): ───@────────@───────iSwap────
                   └──┘   └──────┘
<tf.RaggedTensor [[(-5.596267802565949e-16-3.0908619663705394e-08j), 0j, (1.3510586319685509e-15-3.0908619663705394e-08j), 0j, (-1.810585104067286e-08+0.7071067690849304j), 0j, (-6.181723932741079e-08+0.7071067690849304j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j]]>


In [34]:
def states_distance(state,target):
  diff = state[0] - target[0]
  diff = tf.reshape(diff, [16, 1])
  conjugate_transposed_state = tf.transpose(diff,conjugate=True)
  return tf.tensordot(conjugate_transposed_state, diff, axes = 1)

In [35]:
a = states_distance(state,target_state)
print(a)

tf.Tensor([[2.+0.j]], shape=(1, 1), dtype=complex64)


In [36]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

In [43]:
# gradient(theta[i]) = function(theta)
def custom_gradient(params, target_state):
  param_length = len(params[0])
  grad = [None]*param_length
    
  for i in range(param_length):
      
    params_plus_h = tf.convert_to_tensor([np.concatenate((params[0][0:i],[params[0][i]+h],params[0][i+1:]))])
    params_minus_h = tf.convert_to_tensor([np.concatenate((params[0][0:i],[params[0][i]-h],params[0][i+1:]))])
    state_plus_h = state_layer(qnn, symbol_names=params_names, symbol_values=params_plus_h)
    state_minus_h = state_layer(qnn, symbol_names=params_names, symbol_values=params_minus_h)
    grad[i] = tf.convert_to_tensor((states_distance(state_plus_h,target_state)-states_distance(state_minus_h,target_state))/(2*h))

  return grad

In [40]:
with tf.GradientTape() as g:
  x  = tf.constant([1.0])
  z  = tf.constant([2.0])
  g.watch(x)
  g.watch(z)
  y = x * x + z*z + x*z
gr = g.gradient(y, (x,z))
print(type(gr))
print(gr)

<class 'tuple'>
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.], dtype=float32)>)


In [44]:
circuit_grad = custom_gradient(params, target_state)
print(circuit_grad)
print(type(circuit_grad))

[<tf.Tensor: shape=(1, 1), dtype=complex64, numpy=array([[0.+0.j]], dtype=complex64)>, <tf.Tensor: shape=(1, 1), dtype=complex64, numpy=array([[0.69976425+0.j]], dtype=complex64)>, <tf.Tensor: shape=(1, 1), dtype=complex64, numpy=array([[0.+0.j]], dtype=complex64)>, <tf.Tensor: shape=(1, 1), dtype=complex64, numpy=array([[0.+0.j]], dtype=complex64)>, <tf.Tensor: shape=(1, 1), dtype=complex64, numpy=array([[0.+0.j]], dtype=complex64)>, <tf.Tensor: shape=(1, 1), dtype=complex64, numpy=array([[0.+0.j]], dtype=complex64)>, <tf.Tensor: shape=(1, 1), dtype=complex64, numpy=array([[0.+0.j]], dtype=complex64)>, <tf.Tensor: shape=(1, 1), dtype=complex64, numpy=array([[0.+0.j]], dtype=complex64)>]
<class 'list'>


In [45]:
 adam_optimizer.apply_gradients(zip(circuit_grad, params))

AttributeError: ignored

In [ ]:
tf.tensordot(y, x, axes = 1)

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[14]])>